In [6]:
#shell of notebook

#Company Report Imports
import pandas as pd

import plotly.graph_objects as go

import networkx as nx
import ast
%matplotlib inline  
#from networkx.drawing.nx_agraph import write_dot, graphviz_layout
import matplotlib.pyplot as plt

#Company Suggestions Imports
import re
import sys
from IPython.display import display, HTML

#Display Dropdown Imports
import ipywidgets as widgets
from IPython.display import display
import functools

#Display Button Imports:
import asyncio



# 0. Core/Background Functions

### Display Dropdown

In [7]:
#create the sec codes dataset
#sic_codes = pd.read_csv('corpwatch_api_tables_csv/sic_codes.csv', sep = "\t", escapechar='\\')
#sic_sectors = pd.read_csv('corpwatch_api_tables_csv/sic_sectors.csv', sep = "\t", escapechar='\\')

In [10]:
# sic_sectors_codes = pd.merge(sic_codes,
#                      sic_sectors,
#                      on='sic_sector', 
#                      how='left')x
sic_sectors_codes = pd.read_csv("sector_output.csv")

In [12]:
def on_button_clicked(b, rs_="some_default_string"):
    #test_print = dd.value
    print(rs_)

def valfunc(val):
    print(val)

def display_drop_down(df_column):
    """
    Passes in a Dataframe column to set the dropdown bar. Use in tandem with button code
    to create clickable button to direct users to different functions
    """
    dd = widgets.Dropdown(options= 
          df_column.unique(),
    value=df_column.iloc[0],
    description='Type:',
    disabled=False,
    )
    
    #Code required to display button first time 
    widgets.Button.on_click.__doc__
    
    #Code required to display button 
    button = widgets.Button(description="Go")
    output = widgets.Output()
    
    #displays dropdown and button
    display(dd, button, output) 
    
    button.on_click(functools.partial(on_button_clicked,rs_=dd.value ))
    print('2')
    return dd

#dd = display_drop_down(sic_sectors_codes['sector_group_name']) 

def test():
    print('1')
    df = display_drop_down(sic_sectors_codes['sector_group_name']) 
    print('3')

test()

1


Dropdown(description='Type:', options=(' Agriculture, forestry, and fishing', 'Mining', 'Construction', 'Manuf…

Button(description='Go', style=ButtonStyle())

Output()

2
3


In [13]:
def display_drop_down(df_column):
    """
    Passes in a Dataframe column to set the dropdown bar. Use in tandem with button code
    to create clickable button to direct users to different functions
    """
    dd = widgets.Dropdown(options= 
          df_column.unique(),
    value=df_column.iloc[0],
    description='Type:',
    disabled=False,
    )

    return dd

dd = display_drop_down(sic_sectors_codes['sector_group_name']) #set this to the column you like to define the code 


In [14]:
def unique_non_nan(elem):
    """
    To be used in aggregation functions, rolls elements into a list, discaring nans
    """
    return [a for a in list(pd.unique(elem)) if pd.notna(a)]

def deduplicate(col, ls):
    if col in ls: ls.remove(col)
    return ls

In [15]:
#Column Names 
PARENT_CW_ID = "parent_cw_id"
CHILD_COMPANY_NAMES = "child_company_names"
CHILD_CW_IDS = "child_cw_ids"
PARENT_CW_IDS = "parent_cw_ids"
PARENT_COMPANY_NAMES = "parent_company_names"
CW_ID = 'cw_id'
INDUSTRY_NAME = 'industry_name'
SECTOR_NAME = 'sector_name'
NUM_PARENTS = 'num_parents'
NUM_CHILDREN = 'num_children'
TOP_PARENT_ID = 'top_parent_id'
COMPANY_NAME = 'company_name'
CHILD_CW_ID_LIST = 'child_cw_id_list'
PARENT_CW_ID_LIST = 'parent_cw_id_list'


In [16]:
def create_master_company_data(output_name, corp_watch_dir = "corpwatch"):
    company_info = pd.read_csv('corpwatch_api_tables_csv/company_info.csv', sep = "\t", escapechar='\\',  low_memory=False)
    recent_companies = company_info[company_info['most_recent'] == 1]
    
    #read relationships file which we will use to show parent and child relationships 
    relationships = pd.read_csv('corpwatch_api_tables_csv/relationships.csv', sep = "\t", escapechar='\\')
    relationships_recent = relationships[relationships['year'] > 2018]
    
    #group by the parent cw id, to get a list of children 
    parent_child_groups = relationships_recent.groupby(['parent_cw_id']).agg({
     'parent_cw_id': "first",
     'cw_id': unique_non_nan,
     'clean_company': unique_non_nan})
    
    parent_child_groups['child_cw_id_list'] = parent_child_groups.apply(lambda row: deduplicate(row['parent_cw_id'], row.cw_id), axis = 1)
    #parent_child_groups.rename(index = {'parent_cw_id': "cw_id"}, inplace = False)
    parent_child_groups = parent_child_groups.rename(
    columns= {
    "cw_id": CHILD_CW_IDS,
    'clean_company': CHILD_COMPANY_NAMES})

    parent_child_groups = parent_child_groups[['child_cw_id_list']]
    
    #group by the cw_id to get a list of parents for each cw id 
    child_parent_groups = relationships_recent.groupby(['cw_id']).agg({
     'parent_cw_id' : lambda x:  unique_non_nan(x),
     'clean_company': lambda x: unique_non_nan(x)})
    child_parent_groups['child_cw_id'] = child_parent_groups.index
    child_parent_groups['parent_cw_id_list'] = child_parent_groups.apply(lambda row: deduplicate(row['child_cw_id'], row['parent_cw_id']), axis =1)
   
    company_joined1 = pd.merge(recent_companies,
                     parent_child_groups, 
                     left_on= 'cw_id',
                     right_on='parent_cw_id',          
                     how='left')
    
    company_joined2 =  pd.merge(company_joined1,
                     child_parent_groups, 
                     left_on= 'cw_id',
                     right_on='child_cw_id',
                     how='left')
    company_joined2.to_csv(output_name)

In [17]:
#Run the code to create the company master table 
COMPANY_MASTER_TABLE_NAME = "company_master_table.csv"
#Only need to run this code once 
#create_master_company_data(COMPANY_MASTER_TABLE_NAME)

## List Parse

In [18]:
#required to read the list column from a file 
def list_parse(s):
    strings = str(s).strip("[]").split(",")
    ls = []
    for i in strings:
        try:
            ls.append(int(float(i.strip())))
        except (ValueError):
            ""
    return ls

## Code to read in the company file

In [19]:
#read in company master file, parse, add location information 

def read_company_file():
    data = pd.read_csv("company_master_table.csv", 
                   dtype={"child_cw_id_list" :'str',
                          "parent_cw_id_list": 'str'}, converters={1:ast.literal_eval})
    return data

def clean_company_data(data):
    data[CHILD_CW_ID_LIST] = data[CHILD_CW_ID_LIST].apply(lambda x: list_parse(x))
    data[PARENT_CW_ID_LIST] = data[PARENT_CW_ID_LIST].apply(lambda x: list_parse(x))
    families = data[((data[NUM_PARENTS]> 0) | (data[NUM_CHILDREN] > 0))]  #i might not want to do this
    
    #join with the location 
    location = pd.read_csv('corpwatch_api_tables_csv/company_locations.csv', sep = "\t", escapechar='\\')
    location_filtered = location[location['max_year'] > 1950] #changed by max in the hope we'll get more information
    location_aggregations = {'city': "first",
                         'state': "first",
                         'country_code':  "first"
                         }
    location_agg = location_filtered.groupby('cw_id').agg(location_aggregations)
    
    families_location_Domestic = pd.merge(families, location_agg,
                     left_on='cw_id', 
                     right_on='cw_id',
                     how='left')
    
    #added countries
    countries = pd.read_csv('corpwatch_api_tables_csv/un_countries.csv', sep = "\t", escapechar='\\')
    countries_agg = pd.DataFrame(countries, columns = ['country_code', 'country_name'])

#     countries_aggregations = {'country_code':  "first",
#                               'country_name':  "first"}
    
    #country_agg = countries.groupby('country_code').agg(countries_aggregations)
    
    families_location_International = pd.merge(families_location_Domestic, countries_agg,
                     left_on='country_code', 
                     right_on='country_code',
                     how='left')
    
    #create the sec codes dataset
    #sic_codes = pd.read_csv('corpwatch_api_tables_csv/sic_codes.csv', sep = "\t", escapechar='\\')
#     sic_sectors_codes = pd.merge(sic_codes,
#                      sic_sectors,
#                      on='sic_sector', 
#                      how='left')    
    sic_sectors = pd.read_csv('corpwatch_api_tables_csv/sic_sectors.csv', sep = "\t", escapechar='\\')
    print(sic_sectors.head())

    
    ## get a dataframe containing all granularities of sectors
    families_location = pd.merge(families_location_International,
                     sic_sectors[['sic_sector','sector_group', 'sector_group_name']],
                     on='sic_sector', 
                     how='left')
    families_location.head()
    
    
    #index on cw_id for faster lookup 
    indexed = families_location.set_index(CW_ID) ## should not use the filtered list
    family_groups = families_location.groupby(families_location[TOP_PARENT_ID])
    return indexed, family_groups

In [61]:
company_master_data = read_company_file()
#company_master_data.columns
company_data, family_groups = clean_company_data(company_master_data)

#Used to create sector tables, not needed if you have the file
#company_data = company_data.reset_index()

   sic_sector                         sector_name  sector_group  \
0         100      Agricultural production- crops             1   
1         200  Agricultural production- livestock             1   
2         700               Agricultural services             1   
3         800                            Forestry             1   
4         900      Fishing, hunting, and trapping             1   

                     sector_group_name  
0   Agriculture, forestry, and fishing  
1   Agriculture, forestry, and fishing  
2   Agriculture, forestry, and fishing  
3   Agriculture, forestry, and fishing  
4   Agriculture, forestry, and fishing  


In [62]:
company_master_data.head()

,Unnamed: 0,row_id,cw_id,most_recent,year,cik,irs_number,best_location_id,sic_code,industry_name,...,top_parent_id,company_name,max_year,min_year,no_sic,child_cw_id_list,parent_cw_id,clean_company,child_cw_id,parent_cw_id_list
0,0,18,37,1,2019,3153,630004250.0,74,4911.0,Electric services,...,37,ALABAMA POWER CO,2019,2003,0,"[1665, 993179, 101861, 1018259, 781, 101870, 1...","[12009.0, 22569.0, 781.0, 1197.0, 1665.0]",['Alabama Power Co'],37.0,"[12009, 22569, 781, 1197, 1665]"
1,1,24,45,1,2019,3499,510100517.0,88,6798.0,Real estate investment trusts,...,9019,ALEXANDERS INC,2019,2003,0,"[1038175, 1038176, 1038177, 1038178, 1038179, ...","[14480.0, 9019.0]",['Alexanders Inc'],45.0,"[14480, 9019]"
2,2,39,69,1,2019,4515,131502798.0,137,4512.0,"Air transportation, scheduled",...,105,AMERICAN AIRLINES INC,2019,2003,0,"[964843, 964842, 411832, 411833, 416126, 41613...",[105.0],['American Airlines Inc'],69.0,[105]
3,3,65,120,1,2019,6879,540124790.0,246,4911.0,Electric services,...,75,APPALACHIAN POWER CO,2019,2003,0,"[993634, 834826, 696650, 992937, 136825, 95658...","[956587.0, 1029138.0, 75.0, 927.0, 1345.0, 149...",['Appalachian Power Co'],120.0,"[956587, 1029138, 75, 927, 1345, 1492, 1674]"
4,4,79,130,1,2019,7323,831918668.0,273,4911.0,Electric services,...,1180,ENTERGY ARKANSAS LLC,2019,2003,0,"[899487, 1001712, 1180]",[590084.0],['Entergy Arkansas LLC'],130.0,[590084]


In [100]:
#Sector codes 

companies_by_sector_group = company_data.groupby(['sector_group_name', 'top_parent_id' ])

companies_by_sector_group.head()
t1 = companies_by_sector_group.agg({companies_by_sector_group['company_name']: lambda x : len(pd.unique(x))})
# t2 = t1['cw_id'].groupby(level=0, group_keys=False)
# t2 = t2.apply(lambda x: x.sort_values(ascending=False)).reset_index()
# t2.to_csv("sector_group_name.csv")


# companies_by_sector_name = company_data.groupby(['sector_name', 'top_parent_id' ])
# t_sector_name = companies_by_sector_name['cw_id'].agg(lambda x : len(pd.unique(x))).to_frame()
# final_t_sector_name = t_sector_name['cw_id'].groupby(level=0, group_keys=False)
# final_t_sector_name = final_t_sector_name.apply(lambda x: x.sort_values(ascending=False)).reset_index()
# final_t_sector_name.to_csv("sector_name.csv")
# #final_t_sector_name[final_t_sector_name['sector_name']=='Oil and gas extraction'].head(25)

# companies_by_indutry_name = company_data.groupby(['industry_name', 'top_parent_id' ])
# t_industry_name = companies_by_indutry_name['cw_id'].agg(lambda x : len(pd.unique(x))).to_frame()
# final_t_indutry_name = t_industry_name['cw_id'].groupby(level=0, group_keys=False)
# final_t_indutry_name = final_t_indutry_name.apply(lambda x: x.sort_values(ascending=False)).reset_index()
# final_t_indutry_name.to_csv("industry_name.csv")
#final_t_indutry_name[final_t_indutry_name['industry_name']=='Asset-backed securities'].head(25)


ValueError: cannot copy sequence with size 14056 to array axis with dimension 4370

### Button Code

In [21]:
slider = IntSlider()

async def f():
    for i in range(10):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

NameError: name 'IntSlider' is not defined

## Helper Functions for Company Report Function

In [22]:
#helper function to extract the location from a row as (Country, City)
def create_location_string(row):
    country = row['country_name']
    city = row['city']
    if country == 'nan':
        return ""
    elif str(country) == 'UNITED STATES':
        state = row['state']
        return str(city) + ", " + str(state)
    else:
        return str(city) + ", " + str(country)

In [23]:
import math

def print_grouped_columns(df, column_name, sortby, limit = 100):      
    for (g, data) in df:
        print(g)
        d = data.sort_values(by = sortby)                        
        for (i,c) in d.iterrows():
            print("\t \t \t " + str(c[column_name]) + " " +  create_location_string(c) +" ") 


In [24]:
def get_name_by_id(index_df, cw_id):
    row = index_df.loc[cw_id]
    return row[COMPANY_NAME]

# A. Start Page

This program contains a database of US-based multinational companies, as well as their subsidiaries. This information was sourced by CorpWatch, a San Francisco-based corporation accountability company, using data found in the Security and Exchange Commission (SEC) EDGAR database. 

You can use this program to search for a specific company, or you can search for the most prominent companies in a particular industry or region.

Disclaimer: Because the SEC database is a required only for US Companies, this dataset only contains US-Based multinational companies and their domestic and international subsidiaries. International 

### Workflow
1. Users choose what kind of search they want


### Choices

- Choose a Company (Go to -> __C. Company Query__)

- Choose a Sector (Go to -> __B. Search by Sector__)

- (Optional) Choose from top multinational companies


##### Disclaimer
Only includes US companies

In [25]:
def AStart():
    '''Display start page + options'''
    #B = Sector
    #C = Company
    while True:
        choice = input("Please select an option: ")
        if choice == "B":
            BSector()
        elif choice == "C":
            CCompany()
        else:
            print("sorry I didn't get that, please try again")

## B. Search by Sector

### Workflow
1. Users select Sector Group (dropdown)
1b. Users select Sector (dropdown)
1c. Users select Industry (dropdown)

2. Top Players Results displayed
3. Users select company to view report (dropdown) (Go to -> __D. Company Report Page__)
3b. Users go back (Go to -> __A. Start Page__)

### Features
- Dropdown Widgets: Sector selection
- Top Players Results
    - Groupby: Industries
    - Groupby: Top Parent IDs
- Dropdown Widgets: Company selection



In [ ]:
def BSector():
    '''display sectors options'''
#     display_drop_down(Sector_Groups)
#     display_drop_down(Sectors)
#     display_drop_down(Industries)
    #SIC_choice = "Sector/industry"
    
    #search_by_sector(SIC_choice)
    #displays top 25 results
    
    #company_choice = display_drop_down(Companies)
    #display_company_report = company_choice
    
    #We should change these its hard to read 
    #print("industries\n", sic_sectors_codes['industry_name'].unique() )
    #print("Sector group\n", sic_sectors_codes['sector_group_name'].unique() )
    #print("Sector name\n", sic_sectors_codes['sector_name'].unique() )
    
    sector_var = input("Select:\nA - For getting top comapnies by industries\nB - For getting top comapnies by sector group\nC - For getting top comapnies by sector\n")
    #sector_name = input("Enter the name of the sector or industry")
    sector_name = None
    if sector_var == "A":
        print("Sector name\n", sic_sectors_codes['sector_name'].unique() )
        sector_name = input("Enter the name of the sector")
        sec_df = getTop25Companies('industry_name', sector_name )[['cw_id_x', 'company_name', 'top_parent_id_x']]
        display(sec_df)         
    
#     s_df = getTop25Companies('sector_name', 'Oil and gas extraction' )[['cw_id_x', 'company_name', 'top_parent_id_x']]
#     display(s_df)
                       
    elif sector_var == "B":
        print("Sector group\n", sic_sectors_codes['sector_group_name'].unique() )
        sector_name = input("Enter the name of the sector group")
        sec_df = getTop25Companies('sector_group_name', sector_name )[['cw_id_x', 'company_name', 'top_parent_id_x'] ]
        display(sec_df)
    elif sector_var == "C":
        print("industries\n", sic_sectors_codes['industry_name'].unique() )
        sector_name = input("Enter the name of the industry")
        sec_df = getTop25Companies('sector_name', sector_name )[['cw_id_x', 'company_name', 'top_parent_id_x'] ]
        display(sec_df)                 
                       
    
#     i_df = getTop25Companies('industry_name', 'Asset-backed securities' )[['cw_id_x', 'company_name', 'top_parent_id_x']]
#     display(i_df)
    
#     sec_df = getTop25Companies('sector_group_name', 'Transportation, communications, and utilities' )[['cw_id_x', 'company_name', 'top_parent_id_x'] ]
#     display(sec_df)
    
    
    # setting first name as index column 
#     reset = sec_df.set_index(["top_parent_id"], inplace = True, 
#                     append = True, drop = True)
#     d = reset[COMPANY_NAME].to_dict()
    d = sec_df[COMPANY_NAME].to_dict()
    company_names = dict(map(reversed, d.items()))
    cw_id = display_drop_down_beta(company_names)
    #print("Got id  " + str(cw_id))
    #print("after searching id  " + str(cw_id))
    df = generate_company_report(cw_id)
    group = family_groups.get
    df = create_top_parent_report(group, top_parent, top_parent_name)

    

In [ ]:
# rachel will start
def display_drop_down_beta(options_list):
    """
    displays a drop down for the items in options list, collects choice from the user and returns a string with
    their choice
    
    Requirements: Sector_group_name, Sector_name, Industry_name
    """
    for k in options_list.keys():
        print (k)
    choice = input('Type the item you would like to choose')
    sel  = int(options_list[choice])
    #print(options_list[choice])
    return sel

In [ ]:
#prakar
def getTop25Companies(sectortype, name):
    """
    returns list of a parent companies in that sub sector, ranked by subsidiaries
    """
    topPlayers = pd.read_csv(sectortype + ".csv")
    com_names = pd.merge(topPlayers[topPlayers[sectortype]==name].head(25),
                         company_data,
                         left_on='top_parent_id',
                         right_on = 'cw_id',
                         how='left'
                        )
    
    
    return com_names



## C. Search by Company

### Workflow
1. Users search by Company Name
1b. Users select Sector (dropdown)
1c. Users select Industry (dropdown)

2. Company Results displayed
3. Users select company to view report (dropdown) (Go to -> __D. Company Report Page__)
3b. Users go back (Go to -> __A. Start Page__)

### Features
- Dropdown Widgets: Sector selection
- Weighted Zone Search
    - Number of Children
    - Company Type
    - Sector Type
- Dropdown Widgets: Company selection

In [ ]:
def CCompany():
    
    print('Please input a company you would like to search for')

    company = input('> ').upper()
    sector = input('What sector are you looking for: ')

    print("Searching for relevant companies with names matching " + company + " in sector " + sector)
    
#     MAX_REPEATS = 10
#     repeats = 0
#     company_id = None
#     choice = None

    df = display_company_suggestions(company_data, company, sector)
    
    
#     while(repeats < MAX_REPEATS):
#         print("enter the loop")
#         print("data frame size " + str(df.size))
#         repeats = repeats = repeats + 1
        
    #finds no search results    
    if df.size == 0:
        print("I'm sorry, I wasn't able to find a company with those attributes. Would you like to do next:")
        print("1) Search for another company?")
        CCompany()
        
    #If it finds only one result    
    elif len(df.index) == 1:
        cw_id = df['cw_id'].iloc[0]
        df = generate_company_report(cw_id)
        # This continued logic should be removed 
#         print('What would you like to do now:')
#         print('1) Search for another Company')
#         print('2) Go Back')
#         print('3) Get Company Report')
    elif len(df.index) > 1:
        results= pd.DataFrame(df, columns = ['company_name', 'num_children', 'sector_name', 'weighted_result', 'company_type'])
        display(results)
        print(" ======================================")
        print('Which company would you like to select:')
        d = df[COMPANY_NAME].to_dict()
        company_names = dict(map(reversed, d.items()))
        cw_id = display_drop_down_beta(company_names)
        
        df = generate_company_report(cw_id)
#         if cw_id is not None:
#             if choice == 1:
#                 df = display_company_suggestions(company_data, company, sector)
#             elif choice == 2:
#                 print('Please input a company you would like to search for')
#                 print('Tips for searching:')
#                 print('-You can enter a partial number')
#                 company = input('> ').upper()
#                 sector = input('What sector are you looking for: ')

#             elif choice == 3:
#                 display_breakdown(company_id)
#             else:
#                 print('Sorry I didnt get that, please select another choice')
    return_to_start()

In [129]:

# Rachel has re written this slightly 
import re
import sys
from IPython.display import display, HTML
def display_company_suggestions(df, company, sector=None):
    company_data['company_type'] = company_data['company_name'].apply(lambda x: get_comp_type(x))
    print("Displaying the sugessionts ")
    
    '''
    first test if name entered finds something in the database
    '''
    df['weighted_result'] = 'default'

    df = df[ df['company_name'].str.match(company) ]
    #print(df)
    #print(len(df.index))
    if df.size != 0:
        df['weighted_result'] = df[['num_children','sector_name','company_type']].apply(lambda x: weighted_result(x,sector), axis=1)
        df = df.sort_values(by=['weighted_result'], ascending = False)
    #display(df)
    return df


In [130]:
#### weighting for search results

def weighted_result(x, sector=None):
    #use zone scoring to compute weight of query result    
    child_weight = .5
    sector_weight = .3
    company_weight = .2
    
    #returns positive if there are any children (favors multinational companies)
    child = (0,1)[x['num_children'] > 0]
    #returns positive if the company is an INC or CORP datatype (More likely to be a US multinational)
    company = (0,1)[x['company_type'] == 'INC' or x['company_type'] == 'CORP' or x['company_type'] == None]
    
    #attempts to put sector(if given) into related companies (replace with dropdown if possible)
    try:
        #print(sector.lower())
        #print(sector.lower() in x['sector_name'].lower())
        sect = (0,1)[sector.lower() in x['sector_name'].lower()]
    except:
        sect = 0

    return child_weight * child + sector_weight * sect + company_weight * company


In [131]:
#print(None in company_data['sector_name'].iloc[0])

## D. Company Report Page

### Workflow

1. User is shown page
2. User select another company to view report (dropdown) (Go to -> __D. Company Report Page__)
2b. Users go back (Go to -> __A. Start Page__)


### Features

#### Parent Company
- Relationship Map
- Company Breakdown (% of Company Industries)
    - Groupby: Top Parent ID
    - Groupby: Sector
    - Top Players Ranking
        - Top Players Results
- Foreign/Domestic (% of Domestic vs. International Companies)
    - Groupby: Location
    - Groupby: State
- Dropdown Widgets: Company selection


#### Subsidiary
- Similar Companies
    - Groupby: Top Parent ID
    - Groupby: Sector
- Dropdown Widgets: Company selection

In [132]:
def create_top_parent_report(group_df, company_id, name_string):
    agg_dictionary = {"row_id" : "count",
                  CW_ID: unique_non_nan,
                  COMPANY_NAME: unique_non_nan,
                 }
    COMPANY_COUNT = "company_count"
    #aggregate by industry
    industry_df_group = group_df.groupby(by = "industry_name")
    industry_df = industry_df_group.agg(agg_dictionary)
    industry_df.rename(columns = {"row_id": COMPANY_COUNT}, inplace = True)
    
    #get the top industries in readable way
    #should be cleaned up
    top_industries_df = industry_df.sort_values(by = COMPANY_NAME, ascending=False)
    top_industries = [k for k in top_industries_df[COMPANY_COUNT].to_dict().keys()]
   
    #sectors 
    sectors = group_df.groupby(SECTOR_NAME).count() 
    sectors.rename(columns = {"row_id": COMPANY_COUNT}, inplace = True)
    subsidiaries = group_df[COMPANY_NAME]

    print("Parent company information "  + str(name_string)) 
    locations = group_df.groupby('country_name').agg(agg_dictionary)
    country_codes = [k for k in locations['row_id'].to_dict().keys()]
    if len(country_codes) > 1:
        print (" * Is Multinational") # we can do better ;p
    print("* Has " + str(len(subsidiaries)) + " subsidiaries")
    
    #change to take the top 3 or so industries if they exist 
    print("* Top industries " + str(top_industries))
    
    #It shows basically nothing if we take out Unknown Industry/Sector, so I'm leaving it in 
    industris_clean = industry_df[industry_df.index != "Unknown Industry"]
    # make pie plots 
    
    industris_clean.plot.pie(y = COMPANY_COUNT, legend = False, title = "Subsidiaries By Industry") 
    #industry_df.plot.pie(y = COMPANY_COUNT, legend = False, title = "Subsidiaries By Industry") 

    plt.show()
    
    sectors_clean = sectors[sectors.index != "Unknown Sector"]
    sectors_clean.plot.pie(y = COMPANY_COUNT, legend = False, title = "Subsidiaries by Sector")
    #sectors.plot.pie(y = COMPANY_COUNT, legend = False, title = "Subsidiaries by Sector")

    plt.show()
    
    print("Subsidiaries locations: " + str(set(country_codes)))
    print("==================== SUBSIDIARIES BY INDUSTRY ============================= ")
    print_grouped_columns(industry_df_group, COMPANY_NAME, NUM_CHILDREN, limit = 100)
    
    return industry_df_group
 

In [133]:
#Create a company report for any company 
#prints some information and the parent company report 
def generate_company_report(company_id):
    co_name = get_name_by_id(company_data, company_id)
    row = company_data.loc[company_id]
    industry = row[INDUSTRY_NAME]
    parents = row[PARENT_CW_ID_LIST]
    top_parent = row[TOP_PARENT_ID]
    top_parent_name = get_name_by_id(company_data, top_parent)
    print("Company: " + str(co_name) + "  company_id: " + str(company_id))
    print("Location: " + create_location_string(row))    
    print("=====================")
    print("")
    #size by industry, number of subsidiaries

    if top_parent != company_id:
        if top_parent in parents:
            print("* t is a subsidiary of: " + str(top_parent_name) + " (" + str(top_parent) + ")")
        else:
            print("* is owned by " + str(parents))
            print("* is an indirect subsidiary of  ")
            print("* is a subsidiary of: " + str(top_parent_name) + " " + str(top_parent) + ")")
        #Ask the user if they would like to see the top parent report before showing it? 
        
        if float(row[NUM_CHILDREN]) > 0.0:
            children = row[CHILD_CW_ID_LIST]
            print("* owns " + str(children) + " companies")
    group = family_groups.get_group(top_parent)   
    create_top_parent_report(group, top_parent, top_parent_name)
    
    #wish list 
    #is a multinational
    #ranking by industry
    #number of child locations
    #subsidiaries locations


# ACTUAL PAGE


In [134]:

# Rachel has re written this slightly 
import re
import sys
from IPython.display import display, HTML
def display_company_suggestions(df, company, sector):
    print("Displaying the suggestions")

    df['weighted_result'] = 'default'

    df = df[ df['company_name'].str.match(company) ]

    df['weighted_result'] = df[['num_children','sector_name','company_type']].apply(lambda x: weighted_result(x,sector), axis=1)
    df = df.sort_values(by=['weighted_result'], ascending = False)
    display(df)
    return df


In [135]:
#add a company type column to the dataset
excluded_count = {}
def get_comp_type(row):
    #row['company_name'] = x['company_name'].strip()
    #name = ['company_name'].split()
    name = str(row).split()
    if len(name[-1]) > 4: #takes care of both 1 word company and edge cases where the last section isn't LLC, CORP, INC, etc.)
        if name[-1] not in excluded_count:
            excluded_count[name[-1]] = 1
        else: 
            excluded_count[name[-1]] += 1
        return None 
    else:
        return name[-1].upper()
    
    #return name[-1]
#company_joined2['company_type'] = 'default'
company_data['company_type'] = company_data['company_name'].apply(lambda x: get_comp_type(x))
#for key in excluded_count: #gives us a headcount of what was taken out 
    #print (key, excluded_count[key])

In [136]:
def return_to_start():
    print('Press Enter to return to the main menu')
    input('>')
    main_page()

In [137]:
def main_page():
    print('Welcome to the Multinational Database')
    print('')
    print('Instructions:')
    print('Using the database, you can either search for a specific company or for the largest companies in a sector')
    print('Which method would you like to choose?: ')
    print('A. Search by Company')
    print('B. Search by Sector')
    choice = input('> ')
    if choice == "A":
        CCompany()
#         print('Which method would you like to choose?: ')
#         print('A. Search by Company')
#         print('B. Search by Sector')
#         choice = input('> ')
    elif choice == "B":
        BSector()
#         print('Which method would you like to choose?: ')
#         print('A. Search by Company')
#         print('B. Search by Sector')
#         choice = input('> ')
    else:
        print("sorry I didn't get that, please try again")
    

In [ ]:
def main():
    main_page()
    
main()

Welcome to the Multinational Database

Instructions:
Using the database, you can either search for a specific company or for the largest companies in a sector
Which method would you like to choose?: 
A. Search by Company
B. Search by Sector


>  B
Select:
A - For getting top comapnies by industries
B - For getting top comapnies by sector group
C - For getting top comapnies by sector
 A


Sector name
 ['Agricultural production- crops' 'Agricultural production- livestock'
 'Agricultural services' 'Forestry' 'Fishing, hunting, and trapping'
 'Metal mining' 'Coal mining' 'Oil and gas extraction'
 'Nonmetallic minerals, except fuels' 'General building contractors'
 'Heavy construction contractors' 'Special trade contractors'
 'Food and kindred products' 'Tobacco manufactures'
 'Textile mill products' 'Apparel and other textile products'
 'Lumber and wood products' 'Furniture and fixtures'
 'Paper and allied products' 'Printing and publishing'
 'Chemicals and allied products' 'Petroleum and coal products'
 'Rubber and miscellaneous plastics products'
 'Leather and leather products'
 'Stone, clay, glass, and concrete products' 'Primary metal industries'
 'Fabricated metal products' 'Industrial machinery and equipment'
 'Electrical and electronic equipment' 'Transportation equipment'
 'Instruments and related products'
 'Miscellaneous manufacturing industries' 'Railroads?'
 'L

Enter the name of the sector Agricultural production- crops


,cw_id,company_name,top_parent_id_x
